In [75]:
import numpy as np
import pandas as pd

In [381]:
def sigmoid(x):
    mask = (x > 0)
    positive_out = np.zeros_like(x, dtype='float64')
    negative_out = np.zeros_like(x, dtype='float64')
    
    # 大于0的情况
    positive_out = 1 / (1 + np.exp(-x, positive_out, where=mask))
    # 清除对小于等于0元素的影响
    positive_out[~mask] = 0
    
    # 小于等于0的情况
    expX = np.exp(x,negative_out,where=~mask)
    negative_out = expX / (1+expX)
    # 清除对大于0元素的影响
    negative_out[mask] = 0
    
    return positive_out + negative_out        

In [620]:
class ANN:
    def __init__(self,size):
#         输入层之外的层数,例子为1+2，layNum=3
        self.layerNum=len(size)
        self.size=size
#         i索引W(i+1)矩阵
        self.layer=[]
        self.offset=[]
        self.zlist=[None]*self.layerNum
        self.alist=[None]*self.layerNum
        self.gradient=[None]*(self.layerNum-1)
#         0->sigmaPrime[2]
        self.sigmaPrime=[None]*(self.layerNum-1)
#         0->dcoverdz[2]
        self.delta=[None]*(self.layerNum-1)
        self.predict_res=0
        for i in range(self.layerNum-1):
            self.layer.append(np.random.rand(size[i+1],size[i]))
            self.offset.append(np.matrix(np.random.rand(size[i+1])).transpose())
    def forward_pass(self,invec):
        a=np.matrix(invec).transpose()
        self.alist[0]=a
        for i in range(self.layerNum-1):
            z=self.layer[i]*a+self.offset[i]
            a=sigmoid(z)
            self.zlist[i+1]=z
            self.alist[i+1]=a
            self.sigmaPrime[i]=np.matrix(np.diag(np.multiply(a,1-a).ravel().tolist()[0]))
        self.predict_res=a
        return
    def calcdelta(self,res):
#         只计算0 1（实际上是2,3）
        layer_no=self.layerNum-2
        self.delta[layer_no]=self.sigmaPrime[layer_no]*(self.predict_res-res)
        while layer_no>0:
            self.delta[layer_no-1]=self.sigmaPrime[layer_no-1]*self.layer[layer_no].transpose()*self.delta[layer_no]
            layer_no-=1
        return
    def backPropagation(self,invec,res,lr=1e-3):
        self.forward_pass(invec)
        self.calcdelta(res)
        for i in range(self.layerNum-1):
            self.gradient[i]=self.delta[i]*np.mat(self.alist[i]).transpose()
#         print(self.gradient[1])
        for i in range(self.layerNum-1):
            self.layer[i]-=lr*self.gradient[i]
            self.offset[i]-=lr*self.delta[i]
    def train(self,train_set,lr=1):
        for row in train_set:
            self.backPropagation(row[:-1],row[-1],lr)
    def test(self,invec):
        a=np.matrix(invec).transpose()
        for i in range(self.layerNum-1):
            a=sigmoid(self.layer[i]*a+self.offset[i])
        return float(a)

In [631]:
a=ANN([2,2,1])
a.layer[0]=np.array([[0.1,0.8],[0.4,0.6]])
a.layer[1]=np.array([[0.3,0.9]])
# print(a.layer)
for i in range(1000):
    a.backPropagation(np.array([0.35, 0.9]),0.5,1)
a.test(np.array([0.35, 0.9]))

0.5

In [567]:
dataSet=pd.read_csv('lab4_dataset/train.csv')

In [526]:
def preprocess(dataSet):
    dataSet.dteday=dataSet.dteday.map(date2int)
    return dataSet.values

In [589]:
dataSet=pd.read_csv('lab4_dataset/train.csv')
ds=preprocess(dataSet)[:,1:]
inputNode=ds.shape[1]-1
a=ANN([inputNode,10,1])
# for i in range(10):
#     a.train(ds)
# for row in ds:
#     print([float(x) for x in row])

In [573]:
a.test(ds[1111][:-1])

0.9999999926206831

In [590]:
a.backPropagation(ds[0][:-1],ds[0][-1],1)

[[2.013e+03]
 [1.000e+00]
 [0.000e+00]
 [1.000e+00]
 [0.000e+00]
 [0.000e+00]
 [6.000e+00]
 [0.000e+00]
 [1.000e+00]
 [2.400e-01]
 [2.879e-01]
 [8.100e-01]
 [0.000e+00]]
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[0.9958511]]


In [618]:
np.multiply(np.matrix([1,2,3]),np.matrix([1,2,3]))

matrix([[1, 4, 9]])

In [477]:
def date2int(str):
    return sum([int(x) for x in str.split('/')])